In [47]:
%matplotlib inline
import numpy as np
import pandas as pd
import re as re
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import numpy as np
import pandas as pd
import re as re
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve

import warnings

fdir = '/Users/naveenkumar/Dropbox/OnlineWork/Datasets/FeedBackLoopTesting/'

df = pd.read_csv(fdir + 'Sensorless_drive_diagnosis.txt', header=None, delimiter=' ')

In [48]:
df.head()
df.shape[1]
col_list = []
for c in range(0,df.shape[1]):
    t = 'col' + str(c)
    col_list.append(t)

df.columns = col_list

In [49]:
col_list[len(col_list)-1]

'col48'

In [50]:
df.rename(columns={col_list[len(col_list)-1]:'target'}, inplace=True)
df.columns

Index([u'col0', u'col1', u'col2', u'col3', u'col4', u'col5', u'col6', u'col7',
       u'col8', u'col9', u'col10', u'col11', u'col12', u'col13', u'col14',
       u'col15', u'col16', u'col17', u'col18', u'col19', u'col20', u'col21',
       u'col22', u'col23', u'col24', u'col25', u'col26', u'col27', u'col28',
       u'col29', u'col30', u'col31', u'col32', u'col33', u'col34', u'col35',
       u'col36', u'col37', u'col38', u'col39', u'col40', u'col41', u'col42',
       u'col43', u'col44', u'col45', u'col46', u'col47', u'target'],
      dtype='object')

In [51]:
def roundn(x,n):
    from math import pow
    return round(x*pow(10,n))/pow(10,n)

def columntype(x):
    t = ''
    if is_numeric_dtype(x) == True:
        t = 'numeric'
    else:
        t = 'string'
    
    return t

def unique_value_counts(x,y,cut_off):
    if y <= cut_off:
        return (s2[x].value_counts()).to_dict()
    
    
def replace_DDMMYY_WITH_DDMMYYYY(indate, AddCentury):
    if len(str(indate)) == 8:
        YY = str(indate)[6:8]
        YY_INT = int(YY) + AddCentury
        return str(indate)[0:6] + str(YY_INT)
    else:
        return np.nan
    

def datediff(d1,d2,date_format):
    df = date_format
    a = datetime.strptime(str(d1), df)
    b = datetime.strptime(str(d2), df)
    delta = b - a
    return delta.days



        
        
def ds_summary(s2):
    cmd = pd.DataFrame(s2.columns,columns=['column_name'])
    cmd['TotalRows'] = len(s2)
    cmd['TotalUniqueLevels'] = cmd['column_name'].apply(lambda col:s2[col].nunique())
    cmd['PercentUniqueValues'] = cmd['TotalUniqueLevels']/cmd['TotalRows']
    cmd['TotalNullValues'] = cmd['column_name'].apply(lambda col:s2[col].isnull().sum())
    cmd['PercentNullValues'] = cmd['TotalNullValues']/cmd['TotalRows']
    cmd['IdColFlag'] = cmd['TotalRows'] == cmd['TotalUniqueLevels']
    cmd['DataType'] = cmd['column_name'].apply(lambda col:columntype(s2[col]))
    cmd['UniqueValue'] = cmd[cmd['TotalUniqueLevels'] < 11]['column_name'].apply(lambda col:(s2[col].value_counts()).to_dict())
    cmd['mean'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(s2[col].mean(),2))
    cmd['min'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(s2[col].min(),2))
    cmd['q1'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(s2[col].quantile(q=0.25),2))
    cmd['median'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(s2[col].median(),2))
    cmd['q3'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(s2[col].quantile(q=0.75),2))
    cmd['max'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(s2[col].max(),2))
    return cmd


In [52]:
data_summary = ds_summary(df)
data_summary.to_csv(fdir+'ds_summary.psv', sep = '|' )

In [53]:
data_summary[data_summary['TotalNullValues'] == 0].shape[0]

49

In [54]:
df['target'].value_counts()

11    5319
10    5319
9     5319
8     5319
7     5319
6     5319
5     5319
4     5319
3     5319
2     5319
1     5319
Name: target, dtype: int64

In [56]:
df['y'] = np.where(df['target']== 1, 1, 0)
df['y'].value_counts()

0    53190
1     5319
Name: y, dtype: int64

In [57]:
df.drop(columns=['target'])


,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col39,col40,col41,col42,col43,col44,col45,col46,col47,y
0,-3.014600e-07,8.260300e-06,-1.151700e-05,-2.309800e-06,-1.438600e-06,-2.122500e-05,0.031718,0.031710,0.031721,-0.032963,...,-0.63308,2.96460,8.1198,-1.4961,-1.4961,-1.4961,-1.4996,-1.4996,-1.4996,1
1,2.913200e-06,-5.247700e-06,3.342100e-06,-6.056100e-06,2.778900e-06,-3.752400e-06,0.030804,0.030810,0.030806,-0.033520,...,-0.59314,7.62520,6.1690,-1.4967,-1.4967,-1.4967,-1.5005,-1.5005,-1.5005,1
2,-2.951700e-06,-3.184000e-06,-1.592000e-05,-1.208400e-06,-1.575300e-06,1.739400e-05,0.032877,0.032880,0.032896,-0.029834,...,-0.63252,2.77840,5.3017,-1.4983,-1.4983,-1.4982,-1.4985,-1.4985,-1.4985,1
3,-1.322600e-06,8.820100e-06,-1.587900e-05,-4.811100e-06,-7.282900e-07,4.143900e-06,0.029410,0.029401,0.029417,-0.030156,...,-0.62289,6.55340,6.2606,-1.4963,-1.4963,-1.4963,-1.4975,-1.4975,-1.4976,1
4,-6.836600e-08,5.666300e-07,-2.590600e-05,-6.490100e-06,-7.940600e-07,1.349100e-05,0.030119,0.030119,0.030145,-0.031393,...,-0.63010,4.51550,9.5231,-1.4958,-1.4958,-1.4958,-1.4959,-1.4959,-1.4959,1
5,-9.584900e-07,5.214300e-08,-4.735900e-05,6.453700e-07,-2.304100e-06,5.499900e-05,0.031154,0.031154,0.031201,-0.032789,...,-0.61124,5.83370,18.6970,-1.4956,-1.4956,-1.4956,-1.4973,-1.4972,-1.4973,1
6,-1.783700e-06,1.303600e-06,-3.785100e-05,-7.083900e-06,6.706300e-06,2.207400e-05,0.033805,0.033804,0.033841,-0.033345,...,-0.62501,17.61800,9.0823,-1.4960,-1.4960,-1.4959,-1.4954,-1.4954,-1.4953,1
7,-2.566600e-06,-1.679500e-07,1.483800e-05,-1.598400e-06,8.709200e-07,1.496100e-05,0.031071,0.031071,0.031056,-0.029695,...,-0.63618,12.83800,4.9079,-1.4990,-1.4990,-1.4990,-1.4968,-1.4968,-1.4968,1
8,-5.474000e-06,1.086500e-07,-1.097200e-05,-1.815600e-06,4.757800e-07,2.378300e-05,0.031309,0.031308,0.031319,-0.031344,...,-0.62465,2.98410,6.6883,-1.4955,-1.4955,-1.4955,-1.4947,-1.4947,-1.4947,1
9,2.825600e-06,6.067600e-06,1.182100e-04,-4.347500e-06,1.492300e-06,-2.917100e-06,0.030947,0.030941,0.030823,-0.032171,...,-0.58170,2.05110,8.8939,-1.4963,-1.4963,-1.4963,-1.4943,-1.4943,-1.4943,1


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, log_loss

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

classifiers = [
    KNeighborsClassifier(3),
    SVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
	AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis(),
    LogisticRegression()]

log_cols = ["Classifier", "Accuracy"]
log 	 = pd.DataFrame(columns=log_cols)

sss = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=0)

X = train[0::, 1::]
y = train[0::, 0]

acc_dict = {}

for train_index, test_index in sss.split(X, y):
	X_train, X_test = X[train_index], X[test_index]
	y_train, y_test = y[train_index], y[test_index]
	
	for clf in classifiers:
		name = clf.__class__.__name__
		clf.fit(X_train, y_train)
		train_predictions = clf.predict(X_test)
		acc = accuracy_score(y_test, train_predictions)
		if name in acc_dict:
			acc_dict[name] += acc
		else:
			acc_dict[name] = acc

for clf in acc_dict:
	acc_dict[clf] = acc_dict[clf] / 10.0
	log_entry = pd.DataFrame([[clf, acc_dict[clf]]], columns=log_cols)
	log = log.append(log_entry)

plt.xlabel('Accuracy')
plt.title('Classifier Accuracy')

sns.set_color_codes("muted")
sns.barplot(x='Accuracy', y='Classifier', data=log, color="b")